In [1]:
import pandas as pd
import os

In [2]:
master = pd.read_pickle(os.path.join("..", "all_that_pickle", "master.pickle"))
team_splits = pd.read_pickle(os.path.join("..", "all_that_pickle", "team_splits_mi.pickle"))

In [4]:
master.columns

Index(['firstName', 'lastName', 'pos', 'birthYear', 'birthMon', 'birthDay',
       'birthCountry', 'birthState', 'birthCity'],
      dtype='object')

In [4]:
# Using assign to create a new column 'birthDate', based on a function that produces its values.
master = master.assign(birthDate = pd.to_datetime(
                                        {
                                            'year': master.birthYear,
                                            'month' : master.birthMon,
                                            'day' : master.birthDay
                                        }
))
master = master.drop(columns=['birthYear','birthMon','birthDay'])
master.head(3)

,firstName,lastName,pos,birthCountry,birthState,birthCity,birthDate
playerID,,,,,,,
aaltoan01,Antti,Aalto,C,Finland,NaN,Lappeenranta,1975-03-04
abdelju01,Justin,Abdelkader,L,USA,MI,Muskegon,1987-02-25
abidra01,Ramzi,Abid,L,Canada,QC,Montreal,1980-03-24


In [8]:
master.birthDate.head(2) # dtype is datetime

playerID
aaltoan01   1975-03-04
abdelju01   1987-02-25
Name: birthDate, dtype: datetime64[ns]

In [9]:
master.birthDate[0]  # Timestamp dtype

Timestamp('1975-03-04 00:00:00')

In [11]:
ts = master.birthDate[0]
print(u"{}, {} {} {}".format(ts.day_name(), ts.day, ts.month_name(), ts.year))

Tuesday, 4 March 1975


In [12]:
tsz = ts.tz_localize('America/Toronto')
tsz

Timestamp('1975-03-04 00:00:00-0500', tz='America/Toronto')

In [13]:
tsz.tz_convert('Europe/London')  # Converting to GMT

Timestamp('1975-03-04 05:00:00+0000', tz='Europe/London')

In [37]:
years = master.birthDate.dt.year  # replace year with month & day to get their values.
years.head(2)

playerID
aaltoan01    1975
abdelju01    1987
Name: birthDate, dtype: int64

In [5]:
strings = master.birthDate.dt.strftime("%Y-%m-%d") # Alternatively, dates = pd.to_datetime(strings)
strings.head(3)

playerID
aaltoan01    1975-03-04
abdelju01    1987-02-25
abidra01     1980-03-24
Name: birthDate, dtype: object

In [36]:
strings = strings.str.replace("-", "xx")
strings.head(2)

playerID
aaltoan01    1975xx03xx04
abdelju01    1987xx02xx25
Name: birthDate, dtype: object

In [27]:
try:
    pd.to_datetime(strings)
except Exception as e:
    print(e)

('Unknown string format:', '1975xx03xx04')


In [35]:
parsed = pd.to_datetime(strings, format="%Yxx%mxx%d")
parsed.head(2)

playerID
aaltoan01   1975-03-04
abdelju01   1987-02-25
Name: birthDate, dtype: datetime64[ns]

In [31]:
strings_error = strings.copy()
strings_error.iloc[34] = "xx123"

try:
    pd.to_datetime(strings_error, format="%Yxx%mxx%d")
except Exception as e:
    print(e)
    

time data 'xx123' does not match format '%Yxx%mxx%d' (match)


In [32]:
# If there are some dates that do not conform to the given format then we can use 'coerce' option. This will give out
# NaT values for them. Try not to use the 'ignore' option.

parsed = pd.to_datetime(strings_error, format="%Yxx%mxx%d", errors='coerce')
parsed.iloc[34]

NaT

In [6]:
birth_dates = pd.Series(master.index, index=master.birthDate).sort_index() # always good to sort the index.
birth_dates.head(3)

birthDate
1940-01-27    harpete01
1940-03-22     keonda01
1940-10-03    ratelje01
Name: playerID, dtype: object

In [7]:
born_1980 = birth_dates['1980']  # when dealing with timeseries pandas is clever enough to know you mean the year.
born_1980.head(3)

birthDate
1980-01-02    zalesmi01
1980-01-10    stanara01
1980-01-13    bouckty01
Name: playerID, dtype: object

In [43]:
birth_dates['1980-07']
birth_dates['1980-07-24']
birth_dates['1980-07-20':'1980-07-31']  # both upper & lower dates are included.

birthDate
1980-07-03    miettan01
1980-07-05    huntetr01
1980-07-08    chouier01
1980-07-15    cheecjo01
1980-07-17    millery01
1980-07-19    tanabda01
1980-07-20    arsende01
1980-07-22    kalindm01
1980-07-24    jillsje01
1980-07-24    printda01
1980-07-31    fischji01
Name: playerID, dtype: object

# About timedeltas

In [44]:
diff = birth_dates.index[1] - birth_dates.index[0]
diff

Timedelta('55 days 00:00:00')

In [45]:
birth_dates.index[10] + diff

Timestamp('1945-11-02 00:00:00')

In [48]:
# We can also subtract for the whole time index instead of doing individually.
birth_dates.index - birth_dates.index[0]

TimedeltaIndex([    '0 days',    '55 days',   '250 days',   '755 days',
                 '1182 days',  '1508 days',  '1808 days',  '1829 days',
                 '1887 days',  '1975 days',
                ...
                '19179 days', '19218 days', '19245 days', '19267 days',
                '19283 days', '19294 days', '19308 days', '19406 days',
                '19434 days', '19440 days'],
               dtype='timedelta64[ns]', name='birthDate', length=4627, freq=None)

In [49]:
diff.components

Components(days=55, hours=0, minutes=0, seconds=0, milliseconds=0, microseconds=0, nanoseconds=0)

In [50]:
# We can add and subtract hours and times
diff = diff + '5H' - '10M'
diff

Timedelta('55 days 04:50:00')

In [51]:
# we can round it to days or hours.
diff.round('10D')

Timedelta('60 days 00:00:00')

In [52]:
diff.round('1H')

Timedelta('55 days 05:00:00')

In [53]:
# Creating period.
birth_dates = birth_dates.to_period(freq="D")
birth_dates.index

PeriodIndex(['1940-01-27', '1940-03-22', '1940-10-03', '1942-02-20',
             '1943-04-23', '1944-03-14', '1945-01-08', '1945-01-29',
             '1945-03-28', '1945-06-24',
             ...
             '1992-07-31', '1992-09-08', '1992-10-05', '1992-10-27',
             '1992-11-12', '1992-11-23', '1992-12-07', '1993-03-15',
             '1993-04-12', '1993-04-18'],
            dtype='period[D]', name='birthDate', length=4627, freq='D')

In [54]:
birth_dates.index[0]

Period('1940-01-27', 'D')

In [56]:
# Object of type 'period' have two important attributes: start_time & end_time which represents the bounds of the period
start = birth_dates.index[0].start_time
end = birth_dates.index[0].end_time
print(start)
print(end)

1940-01-27 00:00:00
1940-01-27 23:59:59.999999999


In [57]:
start < (start + pd.Timedelta('5H')) < end

True

In [58]:
start < (start + pd.Timedelta('25H')) < end

False

In [59]:
# selection strategy & atrributes for timestamp & period are practically the same
birth_dates.index.year

Int64Index([1940, 1940, 1940, 1942, 1943, 1944, 1945, 1945, 1945, 1945,
            ...
            1992, 1992, 1992, 1992, 1992, 1992, 1992, 1993, 1993, 1993],
           dtype='int64', name='birthDate', length=4627)

In [60]:
birth_dates['1975'].head(2)

birthDate
1975-01-02    mroziri01
1975-01-03    vyshese01
Freq: D, Name: playerID, dtype: object

In [61]:
birth_dates["1975-01-09"]

birthDate
1975-01-09    eriksan01
1975-01-09     lingda01
Freq: D, Name: playerID, dtype: object

In [8]:
# Resampling or frequency conversion. '1M' -> month frequency. Used mostly for aggregation
birth_dates.resample('1M')

DatetimeIndexResampler [freq=<MonthEnd>, axis=0, closed=right, label=right, convention=start, base=0]

In [10]:
# This is a bit like groupby for time series data. Below is to find how many players were born in one month.
birth_dates.resample('1M').count().head(12)

birthDate
1940-01-31    1
1940-02-29    0
1940-03-31    1
1940-04-30    0
1940-05-31    0
1940-06-30    0
1940-07-31    0
1940-08-31    0
1940-09-30    0
1940-10-31    1
1940-11-30    0
1940-12-31    0
Freq: M, Name: playerID, dtype: int64

In [11]:
birth_dates.resample('2M').count().head()

birthDate
1940-01-31    1
1940-03-31    1
1940-05-31    0
1940-07-31    0
1940-09-30    0
Freq: 2M, Name: playerID, dtype: int64

In [12]:
birth_dates.resample('1Q').count().head()
# Q-DEC

birthDate
1940-03-31    2
1940-06-30    0
1940-09-30    0
1940-12-31    1
1941-03-31    0
Freq: Q-DEC, Name: playerID, dtype: int64

In [13]:
birth_dates.resample('W').count().head(2)

birthDate
1940-01-28    1
1940-02-04    0
Freq: W-SUN, Name: playerID, dtype: int64

In [23]:
from pandas.tseries.offsets import BDay   #Business Day
p = birth_dates.index[2]
print(p)
print(p + BDay(7))

1940-10-03 00:00:00
1940-10-14 00:00:00


In [25]:
birth_dates.shift(1, freq='D').head()  # shift by 1 day.

birthDate
1940-01-28    harpete01
1940-03-23     keonda01
1940-10-04    ratelje01
1942-02-21    esposph01
1943-04-24    esposto01
Name: playerID, dtype: object

In [26]:
team_splits.head()

L  OL    T    W
name          year month                   
Boston Bruins 1980 Apr    2.0 NaN  0.0  1.0
                   Dec    6.0 NaN  1.0  6.0
                   Feb    4.0 NaN  2.0  6.0
                   Jan    4.0 NaN  1.0  9.0
                   Mar    4.0 NaN  3.0  8.0

In [27]:
# We are looking to convert the year and month
# First need to drop the multi-index and create a regular period
team_splits = team_splits.reset_index()
team_splits.head()

,name,year,month,L,OL,T,W
0,Boston Bruins,1980,Apr,2.0,NaN,0.0,1.0
1,Boston Bruins,1980,Dec,6.0,NaN,1.0,6.0
2,Boston Bruins,1980,Feb,4.0,NaN,2.0,6.0
3,Boston Bruins,1980,Jan,4.0,NaN,1.0,9.0
4,Boston Bruins,1980,Mar,4.0,NaN,3.0,8.0


In [31]:
# axis = 1 means rows. Apply that lambda func to all the rows. This will create a series which can be used to create 
# a new column.
string_date = team_splits.apply(lambda x:
                                   pd.Period("{}-{}".format(x.year, x.month)),
                                   axis=1)

team_splits = team_splits.assign(month=string_date)
team_splits = team_splits.drop("year", axis=1)
team_splits.head()

,name,month,L,OL,T,W
0,Boston Bruins,1980-04,2.0,NaN,0.0,1.0
1,Boston Bruins,1980-12,6.0,NaN,1.0,6.0
2,Boston Bruins,1980-02,4.0,NaN,2.0,6.0
3,Boston Bruins,1980-01,4.0,NaN,1.0,9.0
4,Boston Bruins,1980-03,4.0,NaN,3.0,8.0


In [34]:
string_date.head()

0   1980-04
1   1980-12
2   1980-02
3   1980-01
4   1980-03
dtype: object

In [35]:
team_splits = team_splits.set_index(['month'])

In [38]:
# compute each value for each team, aggregated by quarters
quarter_sums = team_splits.groupby('name').resample('Q').sum()
quarter_means = team_splits.groupby('name').resample('Q').mean()
quarter_sums.head()
quarter_means.head()

L   OL   T         W
name          month                              
Anaheim Ducks 2006Q1  4.333333  2.0 NaN  6.333333
              2006Q2  0.000000  2.0 NaN  1.000000
              2006Q3       NaN  NaN NaN       NaN
              2006Q4  2.333333  2.0 NaN  9.333333
              2007Q1  3.333333  1.0 NaN  8.666667

In [39]:
team_splits.to_pickle(os.path.join('..', 'all_that_pickle', 'team_splits_periods.pickle'))
birth_dates.to_pickle(os.path.join('..', 'all_that_pickle', 'birth_dates.pickle'))